In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
import numpy as np
import statistics as stats
import sklearn
import re

pd.set_option('display.max_columns', None)



%matplotlib inline

In [5]:
# Get the csv

df = pd.read_csv('fifa21_train.csv')


In [7]:
# Define a function


def preprocess(data):

    # Fix the weights (as kg)
    Weight = [] 
    for i in data['Weight']:
        Weight.append(str(i).replace('lbs' , ''))

    data['Weight'] = Weight
    data['Weight'] = data['Weight'].astype(np.number)
    data['Weight'] = data['Weight']/2.2046

    
    # Fix the heights (as cm)
    

    cmheight = []
    for item in data['Height']:
        item = str(item).replace('\"', '')
        feetinches = item.split('\'')
        cmheight.append(round(int(feetinches[0]) * 30.48) + int(feetinches[1]) * 2.54)
    data['Height'] = cmheight
        
    # Remove the stars from columns 'IR', 'W/F', and 'SM'
    newvalues = []
    for item in data['IR']:
        item = str(item).replace('★', '')
        item = item.strip()
        newvalues.append(int(item))
    data['IR']=newvalues
  
    newvalues = []
    for item in data['W/F']:
        item = str(item).replace('★', '')
        item = item.strip()
        newvalues.append(int(item))
    data['W/F']=newvalues
    
    newvalues = []
    for item in data['SM']:
        item = str(item).replace('★', '')
        item = item.strip()
        newvalues.append(int(item))
    data['SM']=newvalues
    
  
    # Transform the 'Contract' column to only give the end date of the contract 
    contractend = []
    for item in data['Contract']:
        if 'Free' in item:
            contractend.append('DELETE')
        else:
            item = item.replace(' On Loan', '')
            item = item[-4:]
            contractend.append(int(item))
        
    data['Contract']=contractend

    # Transform the 'loan date end' column to an on loan (yes/no) column
    data['Loan Date End'] = data['Loan Date End'].fillna(0)

    loanyesno = []
    for item in data['Loan Date End']:
        if item == 0:
            loanyesno.append(item)
        else: 
            loanyesno.append(1)

    data['Loan Date End'] = loanyesno
    data = data.rename(columns={'Loan Date End':'On Loan'})

    data['On Loan']
    
    # Remove the plusses and minuses on the last few rows   
    toclean = ['LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'GK']

    superlist = []

    for column in toclean: 
        values = []
        for item in data[column]:
            if item[1] == '-' or item[1] == '+':
                values.append(int(item[0]))
            else: 
                values.append(int(item[:2]))
        superlist.append(values)


    for i in range(len(toclean)):
        data[toclean[i]] = superlist[i]

    # Clean up currency cells by removing € sign and converting 'M' and 'K' to numbers
    
    column = 'Value'
    newvalues = []
    for item in data[column]:
        item = item.replace('€', '')
        if 'K' in item: 
            item = float(item.replace('K', ''))
            item *= 1000
            newvalues.append(int(item))
        elif 'M' in item: 
            item = float(item.replace('M', ''))
            item *= 1000000      
            newvalues.append(int(item))
        else: 
            item = int(item)
            newvalues.append(int(item))
    data[column]=newvalues
    
    
    column = 'Wage'
    newvalues = []
    for item in data[column]:
        item = item.replace('€', '')
        if 'K' in item: 
            item = float(item.replace('K', ''))
            item *= 1000
            newvalues.append(int(item))
        elif 'M' in item: 
            item = float(item.replace('M', ''))
            item *= 1000000      
            newvalues.append(int(item))
        else: 
            item = int(item)
            newvalues.append(int(item))
    data[column]=newvalues
    
    
    column = 'Release Clause'
    newvalues = []
    for item in data[column]:
        item = item.replace('€', '')
        if 'K' in item: 
            item = float(item.replace('K', ''))
            item *= 1000
            newvalues.append(int(item))
        elif 'M' in item: 
            item = float(item.replace('M', ''))
            item *= 1000000      
            newvalues.append(int(item))
        else: 
            item = int(item)
            newvalues.append(int(item))
    data[column]=newvalues
    
    
    # Clean up Hits column (which contains some values with K in them)¶
    newhits = []
    for item in data['Hits']:
        if 'K' in item:
            item = item.replace('K','')
            item = float(item) * 1000
            newhits.append(int(item))
        else: 
            newhits.append(int(item))
    data['Hits'] = newhits

    # Dropping stuff
    
    # Drop columns with irrelevant attributes
    #Some atributes are used to calculate the rating per category, that we already have, so individual attributes are not necessary here

    data.drop(['Name','Crossing','Finishing','Heading Accuracy','Short Passing','Volleys',
           'Dribbling','Curve','FK Accuracy','Long Passing','Ball Control',
           'Acceleration','Sprint Speed','Agility','Reactions','Balance',
          'Shot Power','Jumping','Stamina','Strength','Long Shots',
          'Aggression','Interceptions','Positioning','Vision','Penalties','Composure',
          'Marking','Standing Tackle','Sliding Tackle',
          'GK Diving','GK Handling','GK Kicking','GK Positioning','GK Reflexes'], axis=1)
    

    #Drop the rows with less than 5% of NaN
    data = data[data['Club'].isna()==False]
    data = data[data['Position'].isna()==False]
    data = data[data['Joined'].isna()==False]
    data = data[data['A/W'].isna()==False]
    data = data[data['D/W'].isna()==False]
    data = data[data(['Contract'] == 'DELETE')

    # Transform column 'Joined' to only have a year
    joinednew = []
    for item in data['Joined']:
        joinednew.append(int(item[-4:]))
    data['Joined'] = joinednew
    
    return data

data = preprocess(df)

KeyError: False

In [4]:
y = data['OVA']

X = data.drop(['OVA'], axis = 1)

NameError: name 'data' is not defined

In [13]:
X_num = data.select_dtypes(np.number)
X_cat = data.select_dtypes(object)

In [14]:
X_cat

,Name,Nationality,Club,BP,Position,Team & Contract,foot,Contract,A/W,D/W
0,A. Pasche,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,Right,2020,High,Medium
1,Alan Carvalho,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan",Right,2020,High,Low
2,S. Giovinco,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,Right,2022,High,Medium
3,J. Evans,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,Right,2021,Medium,Medium
4,Y. Demoncy,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,Right,2021,Low,Medium
...,...,...,...,...,...,...,...,...,...,...
11695,I. Ávila,Peru,Melgar FBC,ST,ST,Melgar FBC 2019 ~ 2021,Right,2021,High,Low
11696,B. Böðvarsson,Iceland,Jagiellonia Białystok,LB,LB,Jagiellonia Białystok 2018 ~ 2021,Left,2021,Medium,Medium
11697,G. Gallon,France,ESTAC Troyes,GK,GK,ESTAC Troyes 2019 ~ 2022,Right,2022,Medium,Medium
11698,J. Flores,Chile,CD Antofagasta,RM,LM CAM RM,CD Antofagasta 2019 ~ 2024,Right,2024,High,Medium
